### Feedback robustness

In [1]:
import logging

import torch
from torch import nn
from torch.nn import functional as F
import numpy as np

from learner.models import dnns
from learner.learner import Learner
from data_store.datasets import decoy_mnist, decoy_mnist_CE_augmented
from xil_methods.xil_loss import RRRLoss, RRRGradCamLoss, CDEPLoss, HINTLoss, RBRLoss
import util

# Hyperparameters
EPOCHS = 50
BATCH_SIZE = 256
LR = 0.001
WEIGHT_DECAY = 0.0001
SAVE_LAST = True
VERBOSE_AFTER_N_EPOCHS = 2
SEED = [1, 10, 100, 1000, 10000]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[Using {DEVICE} device]")

[Using cuda device]


Note: we here use wrong instead of arbitrary feedback

In [2]:
methods = ['RRR', 'RRRGradCAM', 'RBR', 'CDEP', 'HINT', 'CE']
for feed in ['wrong', 'incomplete']:
    for n in range(5):
        util.seed_all(SEED[n])
        for meth in methods:
            if meth == 'RRR':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 10
                loss_fn = RRRLoss(reg)
            elif meth == 'RBR':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 1000000
                loss_fn = RBRLoss(reg)
            elif meth == 'RRRGradCAM':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 10
                loss_fn = RRRGradCamLoss(reg)
            elif meth == 'HINT':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, hint_expl=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)
                reg = 10
                loss_fn = HINTLoss(reg, last_conv_specified=True, upsample=True)
            elif meth == 'CE':
                train_dataloader, test_dataloader = decoy_mnist_CE_augmented(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)
                reg = None
                loss_fn = nn.CrossEntropyLoss()
            elif meth == 'CDEP':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 2000000  
                loss_fn = CDEPLoss(reg)
            print(f'{meth}-{feed}-{n}')
            model = dnns.SimpleConvNet().to(DEVICE)
            optimizer = torch.optim.Adam(model.parameters(), lr=LR)
            modelname = f'Robustness-{feed}-DecoyFMNIST-CNN-{meth}--seed={SEED[n]}'
            learner = Learner(model, loss_fn, optimizer, DEVICE, modelname)
            learner.fit(train_dataloader, test_dataloader, EPOCHS, save_last=SAVE_LAST, verbose_after_n_epochs=VERBOSE_AFTER_N_EPOCHS)
            test_acc, _ = learner.score(test_dataloader, learner.base_criterion)
            f.write(f'{meth}-{feed}-{n}: {test_acc}\n')

[Using Seed= 1]
Loading dataset from existing file!
CDEP-wrong-0
Start training...
Epoch 2| accuracy: 46.6%, loss: 0.007849 | Test Error: Acc: 54.2%, Avg loss: 0.007802
Epoch 4| accuracy: 63.0%, loss: 0.004548 | Test Error: Acc: 63.4%, Avg loss: 0.007425
Epoch 6| accuracy: 68.0%, loss: 0.003844 | Test Error: Acc: 65.1%, Avg loss: 0.007323
Epoch 8| accuracy: 70.5%, loss: 0.003506 | Test Error: Acc: 70.7%, Avg loss: 0.007153
Epoch 10| accuracy: 72.1%, loss: 0.003291 | Test Error: Acc: 72.7%, Avg loss: 0.007063
Epoch 12| accuracy: 73.3%, loss: 0.003131 | Test Error: Acc: 74.2%, Avg loss: 0.007016
Epoch 14| accuracy: 74.3%, loss: 0.003006 | Test Error: Acc: 74.6%, Avg loss: 0.006998
Epoch 16| accuracy: 75.2%, loss: 0.002909 | Test Error: Acc: 75.0%, Avg loss: 0.006979
Epoch 18| accuracy: 76.0%, loss: 0.002821 | Test Error: Acc: 75.4%, Avg loss: 0.006943
Epoch 20| accuracy: 76.7%, loss: 0.002727 | Test Error: Acc: 77.6%, Avg loss: 0.006871
Epoch 22| accuracy: 77.6%, loss: 0.002641 | Test Er

Epoch 32| accuracy: 81.8%, loss: 0.002222 | Test Error: Acc: 79.8%, Avg loss: 0.006749
Epoch 34| accuracy: 82.6%, loss: 0.002121 | Test Error: Acc: 78.2%, Avg loss: 0.006801
Epoch 36| accuracy: 83.3%, loss: 0.002046 | Test Error: Acc: 79.0%, Avg loss: 0.006777
Epoch 38| accuracy: 84.2%, loss: 0.001922 | Test Error: Acc: 75.5%, Avg loss: 0.006866
Epoch 40| accuracy: 85.5%, loss: 0.001755 | Test Error: Acc: 76.8%, Avg loss: 0.006840
Epoch 42| accuracy: 86.9%, loss: 0.001576 | Test Error: Acc: 76.7%, Avg loss: 0.006830
Epoch 44| accuracy: 87.8%, loss: 0.001455 | Test Error: Acc: 77.5%, Avg loss: 0.006795
Epoch 46| accuracy: 88.7%, loss: 0.001344 | Test Error: Acc: 74.1%, Avg loss: 0.006907
Epoch 48| accuracy: 88.8%, loss: 0.001375 | Test Error: Acc: 72.8%, Avg loss: 0.006955
Epoch 50| accuracy: 90.2%, loss: 0.001176 | Test Error: Acc: 69.4%, Avg loss: 0.007082
--> Training took 769.836149 seconds!
[Using Seed= 10000]
Loading dataset from existing file!
CDEP-wrong-4
Start training...
Epoch

Epoch 10| accuracy: 76.8%, loss: 0.002484 | Test Error: Acc: 77.0%, Avg loss: 0.006959
Epoch 12| accuracy: 78.7%, loss: 0.002308 | Test Error: Acc: 78.0%, Avg loss: 0.006913
Epoch 14| accuracy: 80.1%, loss: 0.002177 | Test Error: Acc: 77.6%, Avg loss: 0.006896
Epoch 16| accuracy: 81.3%, loss: 0.002070 | Test Error: Acc: 77.6%, Avg loss: 0.006869
Epoch 18| accuracy: 82.1%, loss: 0.001987 | Test Error: Acc: 78.3%, Avg loss: 0.006835
Epoch 20| accuracy: 82.8%, loss: 0.001927 | Test Error: Acc: 79.5%, Avg loss: 0.006790
Epoch 22| accuracy: 83.4%, loss: 0.001868 | Test Error: Acc: 80.0%, Avg loss: 0.006760
Epoch 24| accuracy: 84.0%, loss: 0.001815 | Test Error: Acc: 79.7%, Avg loss: 0.006754
Epoch 26| accuracy: 84.8%, loss: 0.001753 | Test Error: Acc: 79.1%, Avg loss: 0.006755
Epoch 28| accuracy: 85.5%, loss: 0.001663 | Test Error: Acc: 78.7%, Avg loss: 0.006769
Epoch 30| accuracy: 86.4%, loss: 0.001572 | Test Error: Acc: 78.6%, Avg loss: 0.006769
Epoch 32| accuracy: 87.0%, loss: 0.001495 |

In [3]:
methods = ['RRR', 'RRRGradCAM', 'RBR', 'CDEP', 'HINT', 'CE']
for feed in ['wrong', 'incomplete']:
    for n in range(5):
        util.seed_all(SEED[n])
        for meth in methods:
            if meth == 'RRR':
                train_dataloader, test_dataloader = decoy_mnist(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 10
                loss_fn = RRRLoss(reg)
            elif meth == 'RBR':
                train_dataloader, test_dataloader = decoy_mnist(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 100000
                loss_fn = RBRLoss(reg)
            elif meth == 'RRRGradCAM':
                train_dataloader, test_dataloader = decoy_mnist(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 1
                loss_fn = RRRGradCamLoss(reg)
            elif meth == 'HINT':
                train_dataloader, test_dataloader = decoy_mnist(hint_expl=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)
                reg = 100
                loss_fn = HINTLoss(reg, last_conv_specified=True, upsample=True)
            elif meth == 'CE':
                train_dataloader, test_dataloader = decoy_mnist_CE_augmented(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)
                reg = None
                loss_fn = nn.CrossEntropyLoss()
            elif meth == 'CDEP':
                train_dataloader, test_dataloader = decoy_mnist(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 1000000  
                loss_fn = CDEPLoss(reg)
            print(f'{meth}-{feed}-{n}')
            model = dnns.SimpleConvNet().to(DEVICE)
            optimizer = torch.optim.Adam(model.parameters(), lr=LR)
            model = dnns.SimpleConvNet().to(DEVICE)
            optimizer = torch.optim.Adam(model.parameters(), lr=LR)
            modelname = f'Robustness-{feed}-DecoyMNIST-CNN-{meth}--seed={SEED[n]}'
            learner = Learner(model, loss_fn, optimizer, DEVICE, modelname)
            learner.fit(train_dataloader, test_dataloader, EPOCHS, save_last=SAVE_LAST, verbose_after_n_epochs=VERBOSE_AFTER_N_EPOCHS)
            test_acc, _ = learner.score(test_dataloader, learner.base_criterion)
            f.write(f'{meth}-{feed}-{n}: {test_acc}\n')

[Using Seed= 1]
Loading dataset from existing file!
RRR-wrong-0
Start training...
Epoch 2| accuracy: 97.9%, loss: 0.000328 | Test Error: Acc: 97.5%, Avg loss: 0.005979
Epoch 4| accuracy: 99.0%, loss: 0.000152 | Test Error: Acc: 97.7%, Avg loss: 0.005958
Epoch 6| accuracy: 99.3%, loss: 0.000097 | Test Error: Acc: 97.0%, Avg loss: 0.005974
Epoch 8| accuracy: 99.4%, loss: 0.000076 | Test Error: Acc: 96.9%, Avg loss: 0.005977
Epoch 10| accuracy: 99.6%, loss: 0.000062 | Test Error: Acc: 95.7%, Avg loss: 0.006032
Epoch 12| accuracy: 99.7%, loss: 0.000041 | Test Error: Acc: 93.8%, Avg loss: 0.006101
Epoch 14| accuracy: 99.7%, loss: 0.000040 | Test Error: Acc: 94.7%, Avg loss: 0.006064
Epoch 16| accuracy: 99.7%, loss: 0.000041 | Test Error: Acc: 90.9%, Avg loss: 0.006209
Epoch 18| accuracy: 99.8%, loss: 0.000036 | Test Error: Acc: 91.1%, Avg loss: 0.006207
Epoch 20| accuracy: 99.8%, loss: 0.000030 | Test Error: Acc: 91.0%, Avg loss: 0.006213
Epoch 22| accuracy: 99.6%, loss: 0.000058 | Test Err

Epoch 32| accuracy: 95.4%, loss: 0.000956 | Test Error: Acc: 91.2%, Avg loss: 0.006201
Epoch 34| accuracy: 95.8%, loss: 0.000874 | Test Error: Acc: 94.5%, Avg loss: 0.006073
Epoch 36| accuracy: 96.3%, loss: 0.000741 | Test Error: Acc: 95.7%, Avg loss: 0.006022
Epoch 38| accuracy: 96.3%, loss: 0.000729 | Test Error: Acc: 90.6%, Avg loss: 0.006225
Epoch 40| accuracy: 97.0%, loss: 0.000595 | Test Error: Acc: 94.1%, Avg loss: 0.006083
Epoch 42| accuracy: 97.1%, loss: 0.000551 | Test Error: Acc: 94.0%, Avg loss: 0.006093
Epoch 44| accuracy: 97.1%, loss: 0.000549 | Test Error: Acc: 97.3%, Avg loss: 0.005958
Epoch 46| accuracy: 97.7%, loss: 0.000413 | Test Error: Acc: 97.7%, Avg loss: 0.005943
Epoch 48| accuracy: 98.0%, loss: 0.000333 | Test Error: Acc: 97.7%, Avg loss: 0.005944
Epoch 50| accuracy: 98.0%, loss: 0.000316 | Test Error: Acc: 97.8%, Avg loss: 0.005948
--> Training took 415.312129 seconds!
Loading dataset from existing file!
HINT-wrong-0
Start training...
Epoch 2| accuracy: 91.4%,

Epoch 8| accuracy: 97.0%, loss: 0.012490 | Test Error: Acc: 91.5%, Avg loss: 0.006222
Epoch 10| accuracy: 97.1%, loss: 0.012428 | Test Error: Acc: 92.1%, Avg loss: 0.006208
Epoch 12| accuracy: 97.5%, loss: 0.012495 | Test Error: Acc: 93.4%, Avg loss: 0.006148
Epoch 14| accuracy: 97.9%, loss: 0.011110 | Test Error: Acc: 93.3%, Avg loss: 0.006137
Epoch 16| accuracy: 98.1%, loss: 0.009886 | Test Error: Acc: 93.0%, Avg loss: 0.006158
Epoch 18| accuracy: 98.1%, loss: 0.010415 | Test Error: Acc: 92.4%, Avg loss: 0.006174
Epoch 20| accuracy: 98.1%, loss: 0.010729 | Test Error: Acc: 94.2%, Avg loss: 0.006101
Epoch 22| accuracy: 98.2%, loss: 0.010423 | Test Error: Acc: 92.7%, Avg loss: 0.006154
Epoch 24| accuracy: 98.5%, loss: 0.009700 | Test Error: Acc: 92.6%, Avg loss: 0.006162
Epoch 26| accuracy: 98.4%, loss: 0.008575 | Test Error: Acc: 91.7%, Avg loss: 0.006200
Epoch 28| accuracy: 98.4%, loss: 0.008880 | Test Error: Acc: 91.8%, Avg loss: 0.006195
Epoch 30| accuracy: 98.5%, loss: 0.009551 | 

Epoch 38| accuracy: 95.0%, loss: 5139.556053 | Test Error: Acc: 93.5%, Avg loss: 0.006195
Epoch 40| accuracy: 94.8%, loss: 4998.497384 | Test Error: Acc: 92.5%, Avg loss: 0.006244
Epoch 42| accuracy: 94.7%, loss: 5061.765346 | Test Error: Acc: 92.0%, Avg loss: 0.006243
Epoch 44| accuracy: 94.3%, loss: 4873.611773 | Test Error: Acc: 90.2%, Avg loss: 0.006326
Epoch 46| accuracy: 92.9%, loss: 4859.585785 | Test Error: Acc: 89.5%, Avg loss: 0.006353
Epoch 48| accuracy: 92.3%, loss: 4756.228542 | Test Error: Acc: 88.2%, Avg loss: 0.006440
Epoch 50| accuracy: 92.7%, loss: 4599.096415 | Test Error: Acc: 89.3%, Avg loss: 0.006363
--> Training took 338.938811 seconds!
Loading dataset from existing file!
Train set was augmented: X.size= 120000, y.size= 120000
CE-wrong-1
Start training...
Epoch 2| accuracy: 99.0%, loss: 0.000124 | Test Error: Acc: 97.5%, Avg loss: 0.005957
Epoch 4| accuracy: 99.6%, loss: 0.000038 | Test Error: Acc: 93.2%, Avg loss: 0.006126
Epoch 6| accuracy: 99.7%, loss: 0.00004

Epoch 16| accuracy: 99.4%, loss: 0.000088 | Test Error: Acc: 85.2%, Avg loss: 0.006429
Epoch 18| accuracy: 99.9%, loss: 0.000007 | Test Error: Acc: 87.2%, Avg loss: 0.006354
Epoch 20| accuracy: 99.3%, loss: 0.000125 | Test Error: Acc: 86.7%, Avg loss: 0.006374
Epoch 22| accuracy: 99.9%, loss: 0.000013 | Test Error: Acc: 85.2%, Avg loss: 0.006430
Epoch 24| accuracy: 100.0%, loss: 0.000000 | Test Error: Acc: 86.6%, Avg loss: 0.006377
Epoch 26| accuracy: 100.0%, loss: 0.000000 | Test Error: Acc: 86.9%, Avg loss: 0.006365
Epoch 28| accuracy: 100.0%, loss: 0.000000 | Test Error: Acc: 87.1%, Avg loss: 0.006359
Epoch 30| accuracy: 100.0%, loss: 0.000000 | Test Error: Acc: 87.4%, Avg loss: 0.006353
Epoch 32| accuracy: 100.0%, loss: 0.000000 | Test Error: Acc: 87.6%, Avg loss: 0.006348
Epoch 34| accuracy: 100.0%, loss: 0.000000 | Test Error: Acc: 87.6%, Avg loss: 0.006343
Epoch 36| accuracy: 100.0%, loss: 0.000000 | Test Error: Acc: 87.7%, Avg loss: 0.006339
Epoch 38| accuracy: 100.0%, loss: 0.

...

Epoch 46| accuracy: 99.8%, loss: 0.000035 | Test Error: Acc: 98.4%, Avg loss: 0.005909
Epoch 48| accuracy: 99.9%, loss: 0.000020 | Test Error: Acc: 98.2%, Avg loss: 0.005915
Epoch 50| accuracy: 99.9%, loss: 0.000022 | Test Error: Acc: 98.8%, Avg loss: 0.005893
--> Training took 450.163936 seconds!
Loading dataset from existing file!
RRRGradCAM-incomplete-2
Start training...
Epoch 2| accuracy: 94.2%, loss: 0.003372 | Test Error: Acc: 93.8%, Avg loss: 0.006132
Epoch 4| accuracy: 97.7%, loss: 0.000852 | Test Error: Acc: 97.6%, Avg loss: 0.005971
Epoch 6| accuracy: 98.5%, loss: 0.000491 | Test Error: Acc: 98.2%, Avg loss: 0.005941
Epoch 8| accuracy: 98.8%, loss: 0.000410 | Test Error: Acc: 97.9%, Avg loss: 0.005944
Epoch 10| accuracy: 98.9%, loss: 0.000525 | Test Error: Acc: 98.1%, Avg loss: 0.005932
Epoch 12| accuracy: 98.8%, loss: 0.000498 | Test Error: Acc: 97.8%, Avg loss: 0.005945
Epoch 14| accuracy: 98.4%, loss: 0.000991 | Test Error: Acc: 93.5%, Avg loss: 0.006120
Epoch 16| accuracy

Epoch 24| accuracy: 96.1%, loss: 6043.692924 | Test Error: Acc: 93.7%, Avg loss: 0.006141
Epoch 26| accuracy: 95.8%, loss: 5734.787930 | Test Error: Acc: 95.6%, Avg loss: 0.006065
Epoch 28| accuracy: 96.3%, loss: 5526.426547 | Test Error: Acc: 95.8%, Avg loss: 0.006051
Epoch 30| accuracy: 96.3%, loss: 5533.538654 | Test Error: Acc: 96.5%, Avg loss: 0.006030
Epoch 32| accuracy: 96.0%, loss: 5702.273766 | Test Error: Acc: 96.3%, Avg loss: 0.006028
Epoch 34| accuracy: 95.7%, loss: 5960.323782 | Test Error: Acc: 95.6%, Avg loss: 0.006056
Epoch 36| accuracy: 95.1%, loss: 6021.195478 | Test Error: Acc: 94.7%, Avg loss: 0.006102
Epoch 38| accuracy: 94.8%, loss: 6024.999305 | Test Error: Acc: 95.6%, Avg loss: 0.006062
Epoch 40| accuracy: 95.4%, loss: 5943.367315 | Test Error: Acc: 94.1%, Avg loss: 0.006134
Epoch 42| accuracy: 94.9%, loss: 6206.471576 | Test Error: Acc: 95.4%, Avg loss: 0.006067
Epoch 44| accuracy: 95.1%, loss: 6391.206324 | Test Error: Acc: 96.0%, Avg loss: 0.006038
Epoch 46| 

RBR-incomplete-3
Start training...
Epoch 2| accuracy: 98.1%, loss: 0.000237 | Test Error: Acc: 97.5%, Avg loss: 0.005972
Epoch 4| accuracy: 99.0%, loss: 0.000124 | Test Error: Acc: 96.5%, Avg loss: 0.006009
Epoch 6| accuracy: 99.1%, loss: 0.000111 | Test Error: Acc: 96.3%, Avg loss: 0.006018
Epoch 8| accuracy: 99.7%, loss: 0.000039 | Test Error: Acc: 95.8%, Avg loss: 0.006028
Epoch 10| accuracy: 99.7%, loss: 0.000041 | Test Error: Acc: 95.8%, Avg loss: 0.006027
Epoch 12| accuracy: 99.8%, loss: 0.000024 | Test Error: Acc: 94.1%, Avg loss: 0.006090
Epoch 14| accuracy: 99.4%, loss: 0.000085 | Test Error: Acc: 93.4%, Avg loss: 0.006118
Epoch 16| accuracy: 99.6%, loss: 0.000062 | Test Error: Acc: 95.7%, Avg loss: 0.006024
Epoch 18| accuracy: 99.8%, loss: 0.000026 | Test Error: Acc: 94.6%, Avg loss: 0.006068
Epoch 20| accuracy: 99.5%, loss: 0.000081 | Test Error: Acc: 94.3%, Avg loss: 0.006089
Epoch 22| accuracy: 99.7%, loss: 0.000048 | Test Error: Acc: 95.0%, Avg loss: 0.006057
Epoch 24| ac

Epoch 30| accuracy: 99.9%, loss: 0.000030 | Test Error: Acc: 83.6%, Avg loss: 0.006497
Epoch 32| accuracy: 99.9%, loss: 0.000032 | Test Error: Acc: 82.8%, Avg loss: 0.006517
Epoch 34| accuracy: 99.9%, loss: 0.000014 | Test Error: Acc: 82.8%, Avg loss: 0.006520
Epoch 36| accuracy: 99.9%, loss: 0.000036 | Test Error: Acc: 82.3%, Avg loss: 0.006539
Epoch 38| accuracy: 100.0%, loss: 0.000004 | Test Error: Acc: 83.1%, Avg loss: 0.006507
Epoch 40| accuracy: 99.9%, loss: 0.000040 | Test Error: Acc: 81.6%, Avg loss: 0.006577
Epoch 42| accuracy: 99.9%, loss: 0.000028 | Test Error: Acc: 82.2%, Avg loss: 0.006545
Epoch 44| accuracy: 100.0%, loss: 0.000015 | Test Error: Acc: 83.0%, Avg loss: 0.006511
Epoch 46| accuracy: 99.9%, loss: 0.000032 | Test Error: Acc: 81.1%, Avg loss: 0.006593
Epoch 48| accuracy: 99.9%, loss: 0.000029 | Test Error: Acc: 80.7%, Avg loss: 0.006608
Epoch 50| accuracy: 99.9%, loss: 0.000024 | Test Error: Acc: 82.3%, Avg loss: 0.006547
--> Training took 133.048860 seconds!
[Us

Epoch 8| accuracy: 89.3%, loss: 0.001643 | Test Error: Acc: 91.3%, Avg loss: 0.006225
Epoch 10| accuracy: 90.4%, loss: 0.001573 | Test Error: Acc: 91.8%, Avg loss: 0.006211
Epoch 12| accuracy: 91.3%, loss: 0.001443 | Test Error: Acc: 93.8%, Avg loss: 0.006118
Epoch 14| accuracy: 92.3%, loss: 0.001353 | Test Error: Acc: 94.7%, Avg loss: 0.006082
Epoch 16| accuracy: 93.7%, loss: 0.001088 | Test Error: Acc: 94.2%, Avg loss: 0.006093
Epoch 18| accuracy: 94.5%, loss: 0.000930 | Test Error: Acc: 94.5%, Avg loss: 0.006080
Epoch 20| accuracy: 94.9%, loss: 0.000876 | Test Error: Acc: 94.7%, Avg loss: 0.006070
Epoch 22| accuracy: 95.2%, loss: 0.000816 | Test Error: Acc: 96.4%, Avg loss: 0.006000
Epoch 24| accuracy: 95.5%, loss: 0.000789 | Test Error: Acc: 96.2%, Avg loss: 0.006003
Epoch 26| accuracy: 95.2%, loss: 0.000977 | Test Error: Acc: 94.6%, Avg loss: 0.006063
Epoch 28| accuracy: 95.9%, loss: 0.000805 | Test Error: Acc: 95.7%, Avg loss: 0.006017
Epoch 30| accuracy: 96.2%, loss: 0.000722 | 